<a href="https://colab.research.google.com/github/amus-sal/KTH-Data-mining-Frequent-itemsets/blob/main/gradio_ui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
!pip install gradio huggingface-hub==0.25.2

In [3]:
import gradio as gr
from transformers import AutoTokenizer, TextStreamer
from unsloth import FastLanguageModel
import torch

"""
Initialize LoRA-based model using `unsloth`.
"""
# Parameters for LoRA-based model
model_name = "jackma-00/lora_model_1b"
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
FastLanguageModel.for_inference(model)

streamer = TextStreamer(tokenizer, skip_prompt=True)


def respond(message, history, system_message, max_tokens, temperature, top_p):
    """
    Respond function using LoRA-based inference.
    """
    # Prepare the chat history
    messages = [{"role": "system", "content": system_message}]
    for val in history:
        if val[0]:
            messages.append({"role": "user", "content": val[0]})
        if val[1]:
            messages.append({"role": "assistant", "content": val[1]})

    messages.append({"role": "user", "content": message})

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")

    response = ""
    for token in model.generate(
        input_ids=inputs,
        streamer=streamer,
        max_new_tokens=max_tokens,
        use_cache=True,
        temperature=temperature,
        top_p=top_p,
    ):
        response += token
        yield response


demo = gr.ChatInterface(
    respond,
    additional_inputs=[
        gr.Textbox(value="You are a friendly chatbot.", label="System message"),
        gr.Slider(minimum=1, maximum=2048, value=512, step=1, label="Max new tokens"),
        gr.Slider(minimum=0.1, maximum=4.0, value=1.5, step=0.1, label="Temperature"),
        gr.Slider(
            minimum=0.1,
            maximum=1.0,
            value=0.95,
            step=0.05,
            label="Top-p (nucleus sampling)",
        ),
    ],
)

if __name__ == "__main__":
    demo.launch()


ModuleNotFoundError: No module named 'unsloth'